In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from collections import Counter
from typing import Union
# Load the datasets
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Mapping 'Sex' column to binary values: 1 for male, 0 for female
train_df["Sex"] = train_df["Sex"].map({"male": 1, "female": 0})
test_df["Sex"] = test_df["Sex"].map({"male": 1, "female": 0})

# Display basic information for the training and test datasets
def display_basic_info(df: pd.DataFrame, name: str):
    print(f"--- {name} Dataset ---")
    print(f"Shape: {df.shape}")
    print(f"Data Types:\n{df.dtypes}")
    print(f"Missing Values:\n{df.isnull().sum()}")
    print(f"Statistical Summary:\n{df.describe()}")
    print("\nFirst 5 rows:")
    print(df.head())
    print("\n")

# Explore the datasets
display_basic_info(train_df, "Train")
display_basic_info(test_df, "Test")


--- Train Dataset ---
Shape: (891, 12)
Data Types:
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
Missing Values:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
Statistical Summary:
       PassengerId    Survived      Pclass         Sex         Age  \
count   891.000000  891.000000  891.000000  891.000000  714.000000   
mean    446.000000    0.383838    2.308642    0.647587   29.699118   
std     257.353842    0.486592    0.836071    0.477990   14.526497   
min       1.000000    0.000000    1.000000    0.000000    0.420000   
25%     223.500000    0.000000 

In [2]:
train_df=train_df.drop(["PassengerId","Ticket","Cabin"],axis = 1)
test_df=test_df.drop(["PassengerId","Ticket","Cabin"],axis = 1)
print(train_df.columns)
print(test_df.columns)
y=train_df["Survived"]
print("\n\n\n",train_df["Embarked"])

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')
Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')



 0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object


In [3]:
# Create a new feature for total family members aboard (including the passenger)
train_df["FamilySize"] = train_df["SibSp"] + train_df["Parch"] + 1
test_df["FamilySize"] = test_df["SibSp"] + test_df["Parch"] + 1

# Fill missing Age values with the median (to avoid bias from missing data)
train_df["Age"] = train_df["Age"].fillna(train_df["Age"].median())
test_df["Age"] = test_df["Age"].fillna(test_df["Age"].median())


# Categorize passengers into age groups for better generalization
train_df["AgeGroup"] = pd.cut(train_df["Age"], bins=[0, 12, 18, 35, 60, 80],
                              labels=["Child", "Teen", "YoungAdult", "Adult", "Senior"])
test_df["AgeGroup"] = pd.cut(test_df["Age"], bins=[0, 12, 18, 35, 60, 80],
                             labels=["Child", "Teen", "YoungAdult", "Adult", "Senior"])

# Extract title from the Name column
def extract_title(name):
    if "Mr." in name:
        return "Mr."
    elif "Mrs." in name:
        return "Mrs."
    elif "Miss" in name:
        return "Miss"
    elif "Ms." in name:
        return "Ms."
    elif "Master" in name:
        return "Master."
    else:
        return "Unknown"

# Apply title extraction to Name column
train_df["Name"] = train_df["Name"].apply(extract_title)

# Create a feature that flags passengers traveling alone
train_df["IsAlone"] = (train_df["FamilySize"] == 1).astype(int)
test_df["IsAlone"] = (test_df["FamilySize"] == 1).astype(int)


In [4]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, criterion='gini') -> None:
        self.max_depth = max_depth  # Max depth of the tree
        self.min_samples_split = min_samples_split  # Min samples required to split
        self.tree = None  # Final tree structure

    def gini_impurity(self, y: list) -> float:
        if len(y) == 0:
            return 0
        counts = Counter(y)
        total = len(y)
        gini = 1 - sum((count / total) ** 2 for count in counts.values())
        return gini

    def get_thresholds(self, feature_column: list) -> list: 
        # Remove None, NaN, and "Unknown" values from feature_column
        cleaned = [x for x in feature_column if x is not None and pd.notna(x) and x != "Unknown"]

        thresholds = []
        if len(cleaned) == 0:
            return thresholds  # No valid values, return empty list

        # Check if all cleaned values are numeric (int or float)
        if all(isinstance(x, (int, float)) for x in cleaned):
            # For numeric: use midpoints between sorted unique values
            unique_values = sorted(set(cleaned))
            for i in range(len(unique_values) - 1):
                midpoint = (unique_values[i] + unique_values[i + 1]) / 2
                thresholds.append(midpoint)
            return thresholds
        else:
            # For categorical: use unique categories sorted alphabetically
            unique_categories = sorted(set(cleaned))
            return unique_categories

    def split_data(self, passengers_df: pd.DataFrame, survived: list, feature: str, threshold: float) -> tuple:
        # Split data into left and right groups based on feature and threshold
        passengers = passengers_df.to_dict(orient='records')
        left_pass, left_survived = [], []
        right_pass, right_survived = [], []

        if isinstance(threshold, (int, float)):
            for i in range(len(passengers)):
                if passengers[i][feature] <= threshold:
                    left_pass.append(passengers[i])
                    left_survived.append(survived[i])
                else:
                    right_pass.append(passengers[i])
                    right_survived.append(survived[i])
        else:
            for i in range(len(passengers)):
                if passengers[i][feature] == threshold:
                    right_pass.append(passengers[i])
                    right_survived.append(survived[i])
                else:
                    left_pass.append(passengers[i])
                    left_survived.append(survived[i])

        return left_pass, left_survived, right_pass, right_survived

    def information_gain(self, passengers_df: pd.DataFrame, survived: list, feature: str, threshold: float) -> float:
        # Calculate info gain for a specific feature-threshold split
        parent_gini = self.gini_impurity(survived)

        left_pass, left_survived, right_pass, right_survived = self.split_data(
            passengers_df, survived, feature, threshold)

        w_left = len(left_survived) / len(survived)
        w_right = len(right_survived) / len(survived)

        g_left = self.gini_impurity(left_survived)
        g_right = self.gini_impurity(right_survived)

        info_gain = parent_gini - (w_left * g_left + w_right * g_right)
        return info_gain

    def best_split(self, passengers_df: pd.DataFrame, survived: list) -> Union[str, float, float]:
        # Find the best feature and threshold to split on
        best_feature = None
        best_threshold = None
        best_info_gain = -float('inf')

        for feature in passengers_df.columns:
            thresholds = self.get_thresholds(passengers_df[feature].values)
            for threshold in thresholds:
                temp_info = self.information_gain(passengers_df, survived, feature, threshold)
                if temp_info > best_info_gain:
                    best_info_gain = temp_info
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold, best_info_gain

    def fit(self, X: pd.DataFrame, y, depth=0):
        y=list(y)
        # Train the tree recursively

        # Stop if all labels are the same
        if len(set(y)) == 1:
            return {"type": "leaf", "prediction": y[0]} 

        # Stop if tree depth exceeded
        if self.max_depth is not None and depth >= self.max_depth:
            y_list = list(y)
            prediction = max(set(y_list), key=y_list.count)
            return {"type": "leaf", "prediction": prediction}

        # Stop if too few samples to split
        if len(y) < self.min_samples_split:
            y_list = list(y)
            prediction = max(set(y_list), key=y_list.count)
            return {"type": "leaf", "prediction": prediction}

        # Find best feature and threshold
        feature, threshold, gain = self.best_split(X, y)

        # Split data
        left_pass, left_y, right_pass, right_y = self.split_data(X, y, feature, threshold)

        # If any side is empty, make a leaf with majority class
        if len(left_y) == 0:
            right_y_list = list(right_y)
            prediction = max(set(right_y_list), key=right_y_list.count)
            return {"type": "leaf", "prediction": prediction}
        if len(right_y) == 0:
            left_y_list = list(left_y)
            prediction = max(set(left_y_list), key=left_y_list.count)
            return {"type": "leaf", "prediction": prediction}

        # Recursively build left and right branches
        left_pass = pd.DataFrame(left_pass)
        right_pass = pd.DataFrame(right_pass)

        left_branch = self.fit(left_pass, left_y, depth + 1)
        right_branch = self.fit(right_pass, right_y, depth + 1)

        node = {
            "type": "node",
            "feature": feature,
            "threshold": threshold,
            "left": left_branch,
            "right": right_branch
        }

        if depth == 0:
            self.tree = node  # Save the root of the tree
        return node


    def predict_sample(self, x: dict):
        # Predict the label for a single sample
        node = self.tree
        while node["type"] != "leaf":
            feature = node["feature"]
            threshold = node["threshold"]

            if isinstance(threshold, (int, float)):
                if x[feature] <= threshold:
                    node = node["left"]
                else:
                    node = node["right"]
            else:
                if x[feature] == threshold:
                    node = node["right"]
                else:
                    node = node["left"]

        return node["prediction"]

    def predict(self, X: pd.DataFrame):
        # Predict labels for all samples
        return [self.predict_sample(row) for _, row in X.iterrows()]

X=train_df.drop(["Survived"],axis=1)

Accuracy for first 29 height Trees are:

[0.782, 0.76, 0.804, 0.832, 0.827, 0.827, 0.799, 0.788, 0.777, 0.782, 0.793, 0.765, 0.76, 0.771, 0.771, 0.765, 0.765, 0.76, 0.765, 0.765, 0.765, 0.765, 0.765, 0.765, 0.765, 0.765, 0.765, 0.765, 0.765]

![alt text](7b4c980b-388c-4f2e-888b-698413d113f3-1.png)

In [5]:
class RandomForest:
    def __init__(self, n_trees=100, max_depth=None, min_samples_split=2, max_features='sqrt'):
        # Number of trees in the forest
        self.n_trees = n_trees  
        self.max_depth = max_depth  # Maximum depth of each tree
        self.min_samples_split = min_samples_split  # Min samples to split a node
        self.max_features = max_features  # How many features to consider at each split
        self.trees = []  # List to hold all decision trees

    def bootstrap_sample(self, X, y):
        # Create a bootstrap sample (random sample with replacement)
        n_samples = X.shape[0]
        sample_indices = np.random.choice(n_samples, size=n_samples, replace=True)
        X_sample = X.iloc[sample_indices]
        y_sample = y.iloc[sample_indices].tolist() 
        return X_sample, y_sample

    def random_feature_selection(self, total_features):
        # Select a random subset of features to consider at each split
        if self.max_features == 'sqrt':
            num_features = int(sqrt(total_features))
        elif self.max_features == 'log2':
            num_features = int(np.log2(total_features))
        elif isinstance(self.max_features, int):
            num_features = self.max_features
        else:
            num_features = total_features  # Use all features if no valid option
        
        # Randomly pick feature indices without replacement
        selected_features = np.random.choice(total_features, size=num_features, replace=False)
        return selected_features

    def fit(self, X, y):
        # Build the forest by training multiple decision trees
        self.trees = []
        total_features = X.shape[1]

        for _ in range(self.n_trees):
            # Create a new decision tree
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)

            # Get bootstrap sample of data
            X_sample, y_sample = self.bootstrap_sample(X, y)

            # Randomly select features for this tree
            feature_indices = self.random_feature_selection(total_features)
            features = X.columns[feature_indices]

            # Train the tree on the sample with selected features
            tree.fit(X_sample[features], y_sample)

            # Save features used by this tree (needed for prediction)
            tree.features = features

            # Add the trained tree to the forest
            self.trees.append(tree)

    def predict(self, X):
        # Predict class for each sample by majority vote from all trees
        all_tree_predictions = []

        for tree in self.trees:
            # Predict using only features this tree was trained on
            predictions = tree.predict(X[tree.features])
            all_tree_predictions.append(predictions)

        # Transpose to group predictions by sample
        all_tree_predictions = np.array(all_tree_predictions).T

        final_predictions = []

        for sample_predictions in all_tree_predictions:
            # Count votes and pick the most common class
            vote_counts = Counter(sample_predictions)
            majority_vote = vote_counts.most_common(1)[0][0]
            final_predictions.append(majority_vote)

        return final_predictions

    def predict_proba(self, X):
        # Predict probability distribution of classes for each sample
        all_tree_predictions = []

        for tree in self.trees:
            predictions = tree.predict(X[tree.features])
            all_tree_predictions.append(predictions)

        all_tree_predictions = np.array(all_tree_predictions).T

        probability_predictions = []

        for sample_predictions in all_tree_predictions:
            counts = Counter(sample_predictions)
            total_votes = len(sample_predictions)

            # Calculate proportion of votes for each class
            class_probabilities = {cls: count / total_votes for cls, count in counts.items()}
            probability_predictions.append(class_probabilities)

        return probability_predictions


In [6]:
# Loop over the hyperparameters to train and evaluate the model (Already done)
# BEST PARAMETERS FOUND SO FAR: Trees: 50, Max Depth: 4, Min Samples Split: 5 --> Accuracy: 0.8156]

# for n_trees in n_trees_options:
#     for max_depth in max_depth_options:
#         for min_samples_split in min_samples_split_options:
#             rf = RandomForest(
#                 n_trees=n_trees,
#                 max_depth=max_depth,
#                 min_samples_split=min_samples_split,
#                 max_features='sqrt'
#             )
#             rf.fit(X_train, y_train)
#             preds = rf.predict(X_val)
#             accuracy = sum(pred == true for pred, true in zip(preds, y_val)) / len(y_val)
#             print(f"Trees: {n_trees}, Max Depth: {max_depth}, Min Samples Split: {min_samples_split} Accuracy: {accuracy:.4f}") 

In [7]:
# Prepare the full training set (features and labels)
X_train_full = train_df.drop("Survived", axis=1)  # Features
y_train_full = train_df["Survived"]               # Target

# Prepare the test set (make sure it matches training features)
X_test = test_df[X_train_full.columns]

# Initialize the best Random Forest model found from tuning
rf_best = RandomForest(
    n_trees=50,
    max_depth=4,
    min_samples_split=5,
    max_features='sqrt'
)

# Train the model on the entire training dataset
rf_best.fit(X_train_full, y_train_full)

# Make predictions on the test set
test_predictions = rf_best.predict(X_test)

# Print the predictions as a submission format (PassengerId + Prediction)
passenger_ids = pd.read_csv("test.csv")["PassengerId"]
submission_df = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Survived": test_predictions
})

# Display the full submission
print("\n--- Submission Preview ---")
print(submission_df)



--- Submission Preview ---
     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]
